In [1]:
import sys;
sys.path.insert(0, '..')

In [2]:
import copy

## Exercise 1

Use the `grover_optimizer` function defined in listing 11.1 to find the minimum of the function $f(k) = -(k-3)^2 + 3$ where $0 \le k < 8$.

**Answer:**

In [3]:
from algo import grover_circuit

def grover_optimizer(circuit_params,
                     build_circuit, oracle,
                     update_circuit_params, progress, process_outcome,
                     stopping_condition = lambda flow_state: flow_state['failure_count'] > 7,
                     schedule = [0, 1],
                     starting_result = (None, -1)):
    flow_state = {
        'last_good_outcome_results': starting_result,
        'failure_count': 0,
        'initial_circuit_params': copy.deepcopy(circuit_params),
        'circuit_params': circuit_params
    }

    shots = 100

    def update(outcome_results, flow_state):
        flow_state['last_good_outcome_results'] = outcome_results
        flow_state['failure_count'] = 0
        update_circuit_params(outcome_results, flow_state)

    done = False
    counter = 0
    while not done:
        counter += 1
        for r in schedule:
            print('\niteration', r)
            function = build_circuit(flow_state)
            qc = grover_circuit(function, oracle, r)

            result = qc.measure(shots = shots)

            flow_state['last_run_result'] = result

            outcome = max(result['counts'].items(), key = lambda k: k[1])[0]

            outcome_results = process_outcome(outcome, flow_state)

            if progress(outcome_results, flow_state):
                print('progress', outcome_results)

                update(outcome_results, flow_state)
                break
            else:
                flow_state['failure_count'] += 1
                print('failure', outcome_results)

                if stopping_condition(flow_state):
                    print('\nSTOPPING WITH OUTCOME RESULTS', flow_state['last_good_outcome_results'])
                    done = True
                    break

    return flow_state['last_good_outcome_results']

**Parameters**

For `circuit_params` we will use the same parameters as in the example in section 11.2:

In [4]:
n_key = 3
n_value = 6

terms = [(8, [2]), (8, [1]), (5, [0]), (-16, [1, 2]), (-8, [0, 2]), (-4, [0, 1]), (-6, [])]

circuit_params = {'n_key': n_key, 'n_value': n_value, 'terms': terms}

We can also use the same function as in the book for `build_circuit`:

In [5]:
from algo import build_polynomial_circuit

def build_circuit(flow_state):
    return build_polynomial_circuit(flow_state['circuit_params']['n_key'], flow_state['circuit_params']['n_value'], flow_state['circuit_params']['terms'])

This time we are looking for the minimum, so we will use an oracle that tags outcomes with the value 1 in the first digit of the value register:

In [6]:
from algo import oracle_match_1

oracle = oracle_match_1(n_key + n_value, n_key + n_value - 1)

We will change the function `update_circuit_params` to include a new constant that will shift the function up by the new value because we are looking for the minimum (it is subtracted because we expect the best outcome to be negative).

In [7]:
def update_circuit_params(outcome_results, flow_state):
    circuit_params = flow_state['circuit_params']
    k, v = outcome_results
    t = circuit_params['terms']
    t.append((- v, []))
    print('\n------------------------')
    print('New free term:', t[0][0])

We will use the function `progress` to check if the new outcome is less than the previous best outcome (instead of more than like in the example in section 11.2):

In [8]:
def progress(results, state):
    return results[1] < state['last_good_outcome_results'][1] if state['last_good_outcome_results'][1] else True

We can reuse the function `process_outcome` because we are working with the same function as the example in the book:

In [9]:
from util import padded_bin

def process_outcome(outcome, state):
    k = int(padded_bin(n_key + n_value, outcome)[n_value:], 2)
    v = int(padded_bin(n_key + n_value, outcome)[:n_value], 2)
    if v >= 2**(n_value - 1):
        v = v - 2**n_value
    v -= state['circuit_params']['terms'][0][0] - state['initial_circuit_params']['terms'][0][0]
    return (k, v)

Now we are ready to call the `grover_optimizer` function to solve the problem. We will change the parameter `starting_result` to `(None, 0)` because we are looking for the minimum.

In [10]:
grover_optimizer({'n_key': n_key, 'n_value': n_value, 'terms': terms}, build_circuit, oracle, update_circuit_params, progress, process_outcome, starting_result = (None, 0))


iteration 0
progress (7, -13)

------------------------
New free term: 8

iteration 0
failure (7, 0)

iteration 1
failure (0, 7)

iteration 0
failure (2, 15)

iteration 1
failure (2, 15)

iteration 0
failure (5, 12)

iteration 1
failure (7, 0)

iteration 0
failure (5, 12)

iteration 1
failure (4, 15)

STOPPING WITH OUTCOME RESULTS (7, -13)


(7, -13)

## Exercise 2

Pass a different schedule to the `grover_optimizer` to solve the knapsack problem.

**Answer:**

We will use the suggested schedule:

In [11]:
new_schedule = [0, 0, 0, 1, 1, 0, 1, 1, 2, 1, 2, 3, 1, 4, 5, 1, 6, 2, 7, 9, 11, 13, 16, 5, 20, 24, 28, 34, 2, 41, 49, 4, 60]

First, we define all of the parameters for solving the example knapsack problem:

In [12]:
from algo import encode_term
from sim_circuit import *

def build_knapsack_circuit(n_key, n_value, n_weight, v, w):

    key = QuantumRegister(n_key)
    value = QuantumRegister(n_value)
    weight = QuantumRegister(n_weight)
    circuit = QuantumCircuit(key, value, weight)

    for i in range(len(key)):
        circuit.h(key[i])

    for i in range(len(value)):
        circuit.h(value[i])

    for i in range(len(weight)):
        circuit.h(weight[i])

    for (coeff, vars) in v:
        encode_term(coeff, vars, circuit, key, value)

    circuit.iqft(value[::-1], swap=False)

    for (coeff, vars) in w:
        encode_term(coeff, vars, circuit, key, weight)

    circuit.iqft(weight[::-1], swap=False)

    return circuit

In [13]:
n_key = 3
n_value = 3
n_weight = 4

values = [2, 3, 1]
v = [(2, [0]), (3, [1]), (1, [2])]
v_adjusted = [(-max(values) - 0, [])] + v

w = [(3, [0]), (2, [1]), (1, [2])]
max_weight = 4
w_adjusted = [(-item[0], item[1]) for item in w] + [(max_weight, [])]

In [14]:
from algo import oracle_match_0_multi

oracle = oracle_match_0_multi(n_key + n_value + n_weight,
                                  [n_key + n_value - 1, n_key + n_value + n_weight - 1])

In [15]:
def build_circuit(flow_state):
    return build_knapsack_circuit(flow_state['circuit_params']['n_key'], flow_state['circuit_params']['n_value'], flow_state['circuit_params']['n_weight'],
                        flow_state['circuit_params']['v'], flow_state['circuit_params']['w'])

In [16]:
def get_selection_value(selection_binary, v):
    val = 0
    for i in range(len(selection_binary)):
        val += int(selection_binary[::-1][i]) * v[i][0]

    return val


def get_selection_weight(selection_binary, w):
    weight = 0
    for i in range(len(selection_binary)):
        weight += int(selection_binary[::-1][i]) * w[i][0]

    return weight

In [17]:
def process_outcome(outcome, flow_state):
    n = flow_state['circuit_params']['n_key'] + flow_state['circuit_params']['n_value'] + flow_state['circuit_params']['n_weight']
    outcome_selection = padded_bin(n, outcome)[-n_key:]
    outcome_value = get_selection_value(outcome_selection, v)
    outcome_weight = get_selection_weight(outcome_selection, w)
    return outcome_selection, outcome_value, outcome_weight

In [18]:
def progress(results, state):
    outcome_selection, outcome_value, outcome_weight = results
    min_value = state['circuit_params']['min_value']
    return (outcome_value >= min_value) and (outcome_weight <= max_weight)

In [19]:
def action_on_progress(flow_state):
    n_key = flow_state['circuit_params']['n_key']
    n_value = flow_state['circuit_params']['n_value']
    n_weight = flow_state['circuit_params']['n_weight']
    s = flow_state['last_run_result']['state vector']
    # print(knapsack_table_new(s, n_key, n_value, n_weight))

    n = n_key + n_value + n_weight
    ss = sorted(enumerate(s), key=lambda x: int(padded_bin(n, x[0])[-flow_state['circuit_params']['n_key']:], 2))

    print(f'\nLooking for values >= ',{flow_state['circuit_params']['min_value']})

In [20]:
def update_circuit_params(outcome_results, flow_state):
    circuit_params = flow_state['circuit_params']
    outcome_selection, outcome_value, outcome_weight = outcome_results
    v = circuit_params['v']

    v[0] = (-outcome_value - 1, [])

    circuit_params['min_value'] = outcome_value + 1
    action_on_progress(flow_state)

We call `grover_optimizer` with the new schedule:

In [21]:
grover_optimizer({'n_key': n_key, 'n_value': n_value, 'n_weight': n_weight, 'v': v_adjusted, 'w': w_adjusted, 'min_value': 3},
                               build_circuit, oracle, update_circuit_params, progress, process_outcome,
                                stopping_condition = lambda flow_state: flow_state['failure_count'] > 3, schedule=new_schedule)


iteration 0
failure ('001', 2, 3)

iteration 0
failure ('100', 1, 1)

iteration 0
failure ('011', 5, 5)

iteration 1
progress ('110', 4, 3)

Looking for values >=  {5}

iteration 0
failure ('111', 6, 6)

iteration 0
failure ('010', 3, 2)

iteration 0
failure ('010', 3, 2)

iteration 1
failure ('000', 0, 0)

STOPPING WITH OUTCOME RESULTS ('110', 4, 3)


('110', 4, 3)